In [1]:
import torch
from collections import Counter
from collections import defaultdict
from torch_geometric.data import HeteroData

In [ ]:
data = torch.load("../data/graph_BioBERT.pt", weights_only=False)
print(data)

Data(
  x=[3908, 775],
  edge_index=[2, 8360],
  edge_attr=[8360, 13],
  node_names=[3908],
  node_to_idx={
    NDC80=0,
    MACC1-AS1=1,
    DDX58=2,
    TGFbeta1=3,
    SEMA5A=4,
    ABCG1=5,
    MARCKSL1=6,
    MIR874=7,
    LOC105374325=8,
    SETD2=9,
    TUBB3=10,
    NCAPG=11,
    SNHG14=12,
    PROM1=13,
    ISL1=14,
    WDFY3-AS2=15,
    ZXF2=16,
    miR-129=17,
    GOT1=18,
    Chaer=19,
    LINC81507=20,
    ITGAX=21,
    CBL=22,
    LINC00629=23,
    LINC01980=24,
    WNT6=25,
    LINC00673=26,
    LINC00662=27,
    RCC2-AS1=28,
    SIRT2=29,
    FOXP4-AS=30,
    LINC00974=31,
    NONHSAG045500=32,
    TP73=33,
    IPO7=34,
    G6PD=35,
    DAPK1=36,
    DQ786243=37,
    RP5-1120P11.3=38,
    ANXA3=39,
    MIR340=40,
    miR-423-5p=41,
    MIR422A=42,
    CNR2=43,
    miR-199a=44,
    CCNA1=45,
    FMR1=46,
    SUMO1P3=47,
    LINC01128=48,
    DSP=49,
    N-cadherin=50,
    PAK3=51,
    miR-1273e=52,
    LINC00667=53,
    RP11-552M11.4=54,
    BIRC5=55,
    NKX2-1-AS1=56,


In [ ]:
# Create mapping from node indices to node types
node_type_names = ["miRNA", "PCG", "TF", "lncRNA", "snoRNA", "circRNA"]
node_type_ids = torch.argmax(data.x[:, -7:], dim=1)
node_types = [node_type_names[i] for i in node_type_ids.tolist()]

# Count how many nodes belong to each type
node_type_counts = Counter(node_types)

# Print summary of node types
print("Node type counts:")
for t, c in node_type_counts.items():
    print(f"{t}: {c}")

Node type counts:
PCG: 1717
lncRNA: 1333
miRNA: 512
TF: 336
circRNA: 7
snoRNA: 3


In [ ]:
### all nodes are 3908 nodes from all types...

# Group global indices by node type
node_type_to_indices = defaultdict(list)
for i, node_type in enumerate(node_types):
    node_type_to_indices[node_type].append(i)

# Map global node index to local index within each node type
global_to_local = {}
for node_type, indices in node_type_to_indices.items():
    for local_idx, global_idx in enumerate(indices):
        global_to_local[(node_type, global_idx)] = local_idx

In [ ]:
# Initialize HeteroData
hetero_data = HeteroData()

# Add node features for each type
for node_type, indices in node_type_to_indices.items():
    indices_tensor = torch.tensor(indices, dtype=torch.long)
    hetero_data[node_type].x = data.x[
        indices_tensor, :-7
    ]  # Exclude one-hot type vector
    print(
        f"{node_type}: added {len(indices)} nodes with shape {hetero_data[node_type].x.shape}"
    )

PCG: added 1717 nodes with shape torch.Size([1717, 768])
lncRNA: added 1333 nodes with shape torch.Size([1333, 768])
miRNA: added 512 nodes with shape torch.Size([512, 768])
TF: added 336 nodes with shape torch.Size([336, 768])
circRNA: added 7 nodes with shape torch.Size([7, 768])
snoRNA: added 3 nodes with shape torch.Size([3, 768])


In [ ]:
# Build edges per edge type with correct local indices
edge_groups = defaultdict(list)
edge_attrs = defaultdict(list)

for i in range(data.edge_index.shape[1]):
    src = data.edge_index[0, i].item()
    dst = data.edge_index[1, i].item()
    attr = data.edge_attr[i]

    src_type = node_types[src]
    dst_type = node_types[dst]

    edge_type = (src_type, "regulates", dst_type)
    edge_groups[edge_type].append((src, dst))
    edge_attrs[edge_type].append(attr)

In [ ]:
# Add edge_index and edge_attr to HeteroData
for edge_type, edges in edge_groups.items():
    src_type, _, dst_type = edge_type
    src_list = [global_to_local[(src_type, s)] for s, _ in edges]
    dst_list = [global_to_local[(dst_type, d)] for _, d in edges]

    edge_index = torch.tensor([src_list, dst_list], dtype=torch.long)
    edge_attr = torch.stack(edge_attrs[edge_type])

    hetero_data[edge_type].edge_index = edge_index
    hetero_data[edge_type].edge_attr = edge_attr

    print(
        f"Added edge type {edge_type}: {edge_index.shape[1]} edges with attr shape {edge_attr.shape}"
    )

Added edge type ('lncRNA', 'regulates', 'miRNA'): 1088 edges with attr shape torch.Size([1088, 13])
Added edge type ('lncRNA', 'regulates', 'PCG'): 5029 edges with attr shape torch.Size([5029, 13])
Added edge type ('lncRNA', 'regulates', 'TF'): 1570 edges with attr shape torch.Size([1570, 13])
Added edge type ('PCG', 'regulates', 'miRNA'): 11 edges with attr shape torch.Size([11, 13])
Added edge type ('TF', 'regulates', 'lncRNA'): 303 edges with attr shape torch.Size([303, 13])
Added edge type ('lncRNA', 'regulates', 'lncRNA'): 34 edges with attr shape torch.Size([34, 13])
Added edge type ('miRNA', 'regulates', 'lncRNA'): 82 edges with attr shape torch.Size([82, 13])
Added edge type ('PCG', 'regulates', 'PCG'): 36 edges with attr shape torch.Size([36, 13])
Added edge type ('circRNA', 'regulates', 'miRNA'): 3 edges with attr shape torch.Size([3, 13])
Added edge type ('circRNA', 'regulates', 'PCG'): 9 edges with attr shape torch.Size([9, 13])
Added edge type ('PCG', 'regulates', 'lncRNA'

In [ ]:
## 6 node types each node has 768 node features
# 18 edge types each edge has 13 edge attributes.
print(hetero_data)

HeteroData(
  PCG={ x=[1717, 768] },
  lncRNA={ x=[1333, 768] },
  miRNA={ x=[512, 768] },
  TF={ x=[336, 768] },
  circRNA={ x=[7, 768] },
  snoRNA={ x=[3, 768] },
  (lncRNA, regulates, miRNA)={
    edge_index=[2, 1088],
    edge_attr=[1088, 13],
  },
  (lncRNA, regulates, PCG)={
    edge_index=[2, 5029],
    edge_attr=[5029, 13],
  },
  (lncRNA, regulates, TF)={
    edge_index=[2, 1570],
    edge_attr=[1570, 13],
  },
  (PCG, regulates, miRNA)={
    edge_index=[2, 11],
    edge_attr=[11, 13],
  },
  (TF, regulates, lncRNA)={
    edge_index=[2, 303],
    edge_attr=[303, 13],
  },
  (lncRNA, regulates, lncRNA)={
    edge_index=[2, 34],
    edge_attr=[34, 13],
  },
  (miRNA, regulates, lncRNA)={
    edge_index=[2, 82],
    edge_attr=[82, 13],
  },
  (PCG, regulates, PCG)={
    edge_index=[2, 36],
    edge_attr=[36, 13],
  },
  (circRNA, regulates, miRNA)={
    edge_index=[2, 3],
    edge_attr=[3, 13],
  },
  (circRNA, regulates, PCG)={
    edge_index=[2, 9],
    edge_attr=[9, 13],
  },


In [10]:
# show metadata summary (node types and edge types)

print(len(hetero_data.node_types))
print("\nNode types:", hetero_data.node_types)

print(len(hetero_data.edge_types))
print("Edge types:", hetero_data.edge_types)

6

Node types: ['PCG', 'lncRNA', 'miRNA', 'TF', 'circRNA', 'snoRNA']
18
Edge types: [('lncRNA', 'regulates', 'miRNA'), ('lncRNA', 'regulates', 'PCG'), ('lncRNA', 'regulates', 'TF'), ('PCG', 'regulates', 'miRNA'), ('TF', 'regulates', 'lncRNA'), ('lncRNA', 'regulates', 'lncRNA'), ('miRNA', 'regulates', 'lncRNA'), ('PCG', 'regulates', 'PCG'), ('circRNA', 'regulates', 'miRNA'), ('circRNA', 'regulates', 'PCG'), ('PCG', 'regulates', 'lncRNA'), ('PCG', 'regulates', 'TF'), ('lncRNA', 'regulates', 'snoRNA'), ('circRNA', 'regulates', 'TF'), ('TF', 'regulates', 'miRNA'), ('TF', 'regulates', 'PCG'), ('TF', 'regulates', 'circRNA'), ('TF', 'regulates', 'TF')]


In [ ]:
# Choose where to save the file
save_path = "../data/graph_biobert_hetero.pt"

# Save the HeteroData object
torch.save(hetero_data, save_path)

print(f"HeteroData graph saved to: {save_path}")

HeteroData graph saved to: ../data/graph_biobert_hetero.pt
